# 2.2.2 チャットアプリ

In [ ]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv("../.env")

In [ ]:
#1 モデルの定義
llm = ChatOpenAI(model="gpt-4o-mini")

history = []
n = 10
for i in range(10):
    user_input = input("ユーザ入力: ")
    if user_input == "exit":
        break
    #2 HumanMessage の作成と表示
    human_message = HumanMessage(user_input)
    human_message.pretty_print()
    #3 会話履歴の追加
    history.append(HumanMessage(user_input))
    #4 応答の作成と表示
    ai_message = llm.invoke(history)
    ai_message.pretty_print()
    #5 会話履歴の追加
    history.append(ai_message)

# 2.2.3 翻訳アプリ

In [ ]:
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv("../.env")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

#1 プロンプトテンプレートの作成
TRANSLATION_PROMPT = """\
以下の文章を {language} に翻訳し、翻訳結果のみを返してください。
{source_text}
"""
prompt = PromptTemplate.from_template(TRANSLATION_PROMPT)

#2 Runnable の作成
runnable = prompt | llm

language = "日本語"
source_text = """\
cogito, ergo sum
"""

#3 Runnable の実行と結果の表示
response = runnable.invoke(dict(language=language, source_text=source_text))
response.pretty_print()

# 2.2.4 テーブル作成アプリ

In [ ]:
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
import csv

class CSVSaveToolInput(BaseModel):
    filename: str = Field(description="ファイル名")
    csv_text: str = Field(description="CSVのテキスト")

@tool("csv-save-tool", args_schema=CSVSaveToolInput)
def csv_save(filename: str, csv_text: str) -> bool:
    """CSV テキストをファイルに保存する"""
    # parse CSV text
    try:
        rows = list(csv.reader(csv_text.splitlines()))
    except Exception as e:
        return False

    # save to file
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    return True

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [csv_save]
llm_with_tool = llm.bind_tools(tools=tools)

TABLE_PROMPT = """\
{user_input}

結果は CSV ファイルに保存してください。ただし、ファイル名は上記の内容から適切に決定してください。
"""
prompt = PromptTemplate.from_template(TABLE_PROMPT)
get_tool_args = lambda x: x.tool_calls[0]["args"]

runnable = prompt | llm_with_tool | get_tool_args | csv_save

user_input = "フィボナッチ数列の番号と値を10番目まで表にまとめて、CSV ファイルに保存してください。"

In [ ]:
response = runnable.invoke(dict(user_input=user_input))
print(response)